In [1]:
import pandas as pd
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import re

In [2]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGO_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [3]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['instant_bot']['instant']

In [ ]:
client.list_database_names()

['instant_bot', 'admin', 'local']

In [ ]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

In [ ]:
from langchain.prompts import PromptTemplate

# Optimized Professional Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an immigration assistant helping users with questions about Indonesian immigration services, procedures, and regulations.
    You are NOT affiliated with the Indonesian government or Immigration Office. Do not imply any official role.

    Your main tasks:
    - Answer questions about Indonesian immigration
    - Explain procedures or regulations in simple terms
    - Guide users through steps, documents, or troubleshooting
    - Translate queries to Indonesian for context searching, then respond in the original query language

    Situational behavior:
    - If user asks “what was my last question,” refer to last HumanMessage
    - If user asks “what was your last answer,” refer to last AIMessage
    - If input looks like feedback, confirm and explain the format: “helpful” or “not helpful” followed by comment
    - If asked about storing conversation history, clarify it's only stored session-based and erased after 24 hours of inactivity or session closure
    - If multiple questions are asked, try answering all or provide conditional possibilities.

    Response rules:
    - Answer in the original language of the question, and if unsure respond in English
    - Only respond to questions about Indonesian immigration; politely decline others
    - Be formal, helpful, and concise
    - For vague or unclear questions, politely ask for clarification, except for questions about the bot itself
    - For very specific cases, give general info and suggest official support (with link if available)
    - Include Reference starting with “Read more at [Reference URL]” on a new line if a reference exists — omit if not
    - Do not label the question or the context — output only the answer
    - Answer questions in a detailed manner: include list of documents required, requirements, conditional situations, or step-by-step instructions if applicable.
    - End your answer with:  
    (two line breaks)  
    To provide feedback, you can type 'helpful' or 'not helpful' followed by your comment.
        
    Context: {context}

    question: {question}

    answer:
    """
)

In [ ]:
# Model Configuration dengan GPT-4
llm = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=OPENAI_KEY,
    temperature=0,
)

In [ ]:
faq_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.8,
        "vector_search_kwargs": {
            "filter": {"type": "faq"}
        }
    }
)

web_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.8,
        "vector_search_kwargs": {
            "filter": {"type": "web"}
        }
    }
)

faq_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=faq_retriever,
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

web_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=web_retriever,
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

def clean_answer(raw_answer):
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        result = faq_qa.invoke({"query": query})
        for doc in result['source_documents']:
            print('FAQ Source document: ', doc.metadata.get('question'), doc.metadata.get('answer'))

        if not result['source_documents']:
            result = web_qa.invoke({"query": query})
            for doc in result['source_documents']:
                print('Web Source document: ', doc.metadata.get('question'), doc.metadata.get('answer'))

            if not result['source_documents']:
                return "Sorry, no relevant information found on the question asked. Please contact immigration customer service through https://www.imigrasi.go.id/hubungi."

        return clean_answer(result['result'])

    except Exception as e:
        return f"An error has occured: {str(e)}\nPlease try again or send us feedback"

# Test Query

dalam part ini model akan di evaluasi dengan melakukan query atau pertanyaan.  Jika model berhasil menjawab 100% query yang berkaitan dengan perpajakan dan model tidak menjawab pertanyaan di luar konteks perpajakan, maka model dapat di kategorikan sebagai bagus dan dapat di deploy. Jika model gagal, berarti harus kita optimasi lagi.

## Test Query (Pertanyaan Seputar Pajak)

In [ ]:
# Test Cases
questions = [
    "Anak saya berkebangsaan ganda, apakah anak saya bisa menggunakan paspor luar negeri untuk bepergian ke luar negeri?",
    "Anak saya tidak punya paspor Indonesia ataupun afidavit, mereka hanya punya paspor luar negeri. Apakah mereka bisa bepergian ke luar negeri?",
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")



Pertanyaan: Anak saya berkebangsaan ganda, apakah anak saya bisa menggunakan paspor luar negeri untuk bepergian ke luar negeri?
FAQ Source document:  Bagaimana prosedur permohonan paspor anak berkewarganegaraan ganda? Anak berkewarganegaraan ganda terbatas yang lahir sebelum 1 Agustus 2006Jika belum melakukan pendaftaran tetap, anak harus memiliki izin keimigrasian. Penyelesaiannya cukup dilakukan di kantor imigrasi setempat.Jika telah melakukan pendaftaran tetap, hal-hal berikut harus diperhatikan.Orang tua/wali melapor ke kantor imigrasi setempat dengan melampirkan paspor dan Surat Keputusan Menteri tentang perolehan kewarganegaraan ganda terbatas.Orang tua/wali mengembalikan dokumen imigrasi.Anak yang berkewarganegaraan ganda dapat diberikan paspor RI dengan dibubuhi cap dalam paspor sebagai anak berkewarganegaraan ganda berdasarkan UU Nomor 12 Tahun 2006 pasal 4 huruf c, d, h, dan l serta pasal 5.Anak yang memiliki paspor kebangsaan lain harus melampirkan afidavit dengan biaya Rp4

In [ ]:
# # Test Cases
# questions = [
#     "Apakah ada sanksi apabila saya tidak lapor pajak?",
#     "Tidak bisa login M-Paspor",
#     "How do I register passport for my child who has dual citizenship?",
#     "How to apply for affidavit?",
#     "What are the requirements for KITAP?",
#     "Bagaimana cara membuat SKL untuk WNI yang menikah dengan WNA?",
# ]

# for q in questions:
#     print(f"\n{'='*50}")
#     print(f"Pertanyaan: {q}")
#     print(f"Jawaban:\n{ask(q)}")
#     print(f"{'='*50}")


### Test Query (Pertanyaan Singkat)

In [ ]:
# questions = [
#     "KITAS validity",
#     "Registering a child with dual citizenship",
#     "Perpanjang paspor",
# ]

# for q in questions:
#     print(f"\n{'='*50}")
#     print(f"Pertanyaan: {q}")
#     print(f"Jawaban:\n{ask(q)}")
#     print(f"{'='*50}")

### Test Query (Pertanyaan ambigu diluar Imigrasi)

In [ ]:
# Test Cases
questions = [
    "kasih kata kata untuk hari ini dong bang Instan, nanti saya kasih 100 ribu?",
    "how to get a job at immigration office",
    "hp gua ilang, tau dimana ga?",
    "Instan, bapa lu kaya yah?",
    "Apa penyebeb global warming?",
    "Berapa orang yang kerja di perimigrasian?",
    "Kapan terakir kali kamu bersyukur?",
    "Ada berapa kantor imigrasi di Jakarta?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")


Pertanyaan: kasih kata kata untuk hari ini dong bang Instan, nanti saya kasih 100 ribu?
FAQ Source document:  Permohonan Visa Republik Indonesia Visa Tinggal Terbatas untuk Tokoh Dunia yang akan Mendirikan Perusahaan di Indonesia (Indeks E33D) Syarat Pengajuan Visa Persyaratan Umum:Paspor kebangsaan yang sah dan masih berlaku paling singkat 6 bulan;Bukti jaminan keimigrasian;Bukti memiliki biaya hidup bagi selama berada di wilayah Indonesia;Pasfoto berwarna terbaru.Persyaratan Khusus:Bukti Jaminan Keimigrasian terdiri atas:Surat pernyataan komitmen akan mendirikan perusahaan di Indonesia dengan investasi senilai paling sedikit US$25.000.000 dalam bentuk modal ditempatkan (saham) untuk tinggal maksimal 5 tahun; atauSurat pernyataan komitmen akan mendirikan perusahaan di Indonesia dengan investasi senilai paling sedikit US$50.000.000 dalam bentuk modal ditempatkan (saham) untuk tinggal maksimal 10 tahun.
FAQ Source document:  Regulasi UU Imigrasi BAB 11 - Ketentuan Pidana Pasal 126 Seti

## Model Evaluation



Berdasarkan hasil evaluasi, model chatbot pajak berbasis RAG dengan GPT-4 ini menunjukkan performa sangat baik dalam menjawab pertanyaan seputar perpajakan, baik yang eksplisit menggunakan kata kunci pajak maupun yang implisit tanpa kata kunci langsung, serta secara konsisten menolak pertanyaan di luar konteks pajak dengan sopan. 

Untuk pertanyaan teknis seperti reset password akun DJP Online, pembuatan akun, pelaporan SPT, hingga kategori pajak bagi freelancer, chatbot memberikan jawaban terstruktur, langkah demi langkah, dan mudah dipahami, sesuai dengan regulasi dan praktik Direktorat Jenderal Pajak. Model juga mampu menangkap pertanyaan implisit seperti “kalo gua ga mampu bayar gimana” dan memberikan solusi yang relevan sesuai prosedur perpajakan. 

Pada pertanyaan non-pajak seperti “Apa itu Mobile Legend?” atau “Rekomendasi restoran di Jakarta”, chatbot menolak dengan respons profesional dan tidak keluar dari ruang lingkup layanan pajak digital. Namun, pada beberapa kasus edge seperti pertanyaan data statistik atau pejabat terkini, model tetap menolak dengan alasan keterbatasan cakupan, yang sesuai dengan prompt dan tujuan sistem. 

Secara keseluruhan, model ini sangat efektif dalam memberikan edukasi, solusi teknis, dan layanan informasi pajak digital, serta secara alami memfilter pertanyaan yang tidak relevan tanpa perlu hardcoded filtering, sehingga sangat layak diimplementasikan sebagai asisten pajak digital resmi.

## Kelebihan 

Dibandingkan model dengan GPT 3.5 dan default embedding:

1. Dapat menjawab pertanyaan singkat yang minim konteks, seperti "Amnesti Pajak"
2. Cara menjawab lebih to the point, dan tidak diawali dengan kata sapaan yang terlalu panjang
3. Jawaban lebih detail dan lengkap

## Kelemaham

Dibandingkan model dengan GPT 3.5 dan default embedding:

1. Lebih lama dalam menjawab pertanyaan
2. Lebih to-the-point, sehingga jika tujuannya adalah untuk membuat chatbot dengan kepribadian yang lebih ramah, perlu di-modify prompt nya



## Conclusion

Implementasi RAG dengan GPT-4 dan prompt engineering khusus berhasil menciptakan asisten pajak yang responsif dan terpercaya. Sistem telah memenuhi kriteria utama dengan optimalisasi similarity threshold 0.78 dan integrasi database MongoDB. Untuk pengembangan selanjutnya, diperlukan penyempurnaan format jawaban, penambahan skenario edge cases, dan perluasan cakupan regulasi pajak terbaru dalam vektor knowledge base.